## Imports

In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Input, Conv2D, Flatten
from keras.layers import GaussianDropout, GaussianNoise, Reshape

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Constants

## Compile Model

In [4]:
inputs = Input(shape=(11, 144))
reshape = Reshape((11, 144, 1))(inputs)
c1 = Conv2D(30, (3, 3), activation='elu')(reshape)
gd1 = GaussianDropout(0.1)(c1)
gn1 = GaussianNoise(0.1)(gd1)

c2 = Conv2D(30, (1, 35), activation='elu')(gn1)
gd2 = GaussianDropout(0.1)(c2)
gn2 = GaussianNoise(0.1)(gd2)

c3 = Conv2D(30, (7, 1), activation='elu')(gn2)
gd3 = GaussianDropout(0.1)(c3)
gn3 = GaussianNoise(0.1)(gd3)

# y-on
c4 = Conv2D(10, (3, 3), activation='elu')(gn3)
gd4 = GaussianDropout(0.5)(c4)
gn4 = GaussianNoise(0.1)(gd4)
f1 = Flatten()(gn4)
yOn = Dense(88, input_dim=1060,  activation='sigmoid')(f1)

# y-frm
c5 = Conv2D(10, (3, 3), activation='elu')(gn3)
gd5 = GaussianDropout(0.5)(c5)
gn5 = GaussianNoise(0.1)(gd5)
f2 = Flatten()(gn5)
yFrm = Dense(88, input_dim=1060,  activation='sigmoid')(f2)

# y-off
c6 = Conv2D(10, (3, 3), activation='elu')(gn3)
gd6 = GaussianDropout(0.5)(c6)
gn6 = GaussianNoise(0.1)(gd6)
f3 = Flatten()(gn6)
yOff = Dense(88, input_dim=1060,  activation='sigmoid')(f3)

adsr_conv = Model(inputs=inputs, outputs=[yOn, yFrm, yOff])

In [5]:
adsr_conv.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 11, 144)      0                                            
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 11, 144, 1)   0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 9, 142, 30)   300         reshape_3[0][0]                  
__________________________________________________________________________________________________
gaussian_dropout_9 (GaussianDro (None, 9, 142, 30)   0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
gaussian_n